In [1]:
import sys
sys.path.append('..')

import constants.file_handler_constants as fh
from constants.user_constants import *
from constants.attraction_constants import *

import os
import glob
import time
import pandas as pd
import numpy as np
import json
import requests
import ast

from sklearn.neighbors import NearestNeighbors

In [2]:
# get a user data (use mockup data untill there's user api)
user_tag_score_vector = np.array(MOCK_USER[1])

# get attractions data
attraction_tag_score_data = []
attraction_ref = []
try:
    API_ENDPOINT = f"{os.environ['TRIPWEAVER_API']}/api/attraction/getAllData"
    res_all_attractions = requests.post(url=API_ENDPOINT).json()
    
    for cur_attraction in res_all_attractions['attractions']:
        cur_tag_score = cur_attraction['attractionTag']['attractionTagFields']
        cur_ref = (cur_attraction['_id'], cur_attraction['name'])

        attraction_tag_score_data.append(list(cur_tag_score.values()))
        attraction_ref.append(cur_ref)
        
except Exception as e:
    print("get all attraction data failed !")
    print(e)

for cur_tag, cur_ref in zip(attraction_tag_score_data, attraction_ref):
    print(cur_ref)
    print(cur_tag)



get all attraction data failed !
HTTPConnectionPool(host='localhost', port=3000): Max retries exceeded with url: /api/attraction/getAllData (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D565CD3A10>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))


In [3]:
# train KNN Model
attraction_tag_score_matrix = np.array(attraction_tag_score_data.copy())
knn = NearestNeighbors(n_neighbors=20, metric='cosine') # euclidean
knn.fit(attraction_tag_score_matrix)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [10]:
# find top recommendations
user_tag_score_vector = user_tag_score_vector.reshape(1, -1)
distances, indices = knn.kneighbors(user_tag_score_vector)

# Print the recommendations
print("Top Recommendations:")
for i, idx in enumerate(indices[0]):
    print(f"{i+1}. {attraction_ref[idx][1]} (Similarity: {1 - distances[0][i]:.2f})")

Top Recommendations:
1. อาร์ทสเปซ แกลลอรี่ แอนด์มิวสิค บาร์ (Similarity: 1.00)
2. 75 Stang Art Studio and Reggae Bar (Similarity: 0.93)
3. Bless Art Gallery (Similarity: 0.88)
4. Beloved tattoo (Similarity: 0.88)
5. หอศิลป์กำพล (Similarity: 0.86)
6. Joker Tattoo Patong (Similarity: 0.84)
7. Easy Pole dance (Similarity: 0.83)
8. Olivia Nail Studio (Similarity: 0.82)
9. Colizeum Cyber Esport (Similarity: 0.79)
10. ภูเก็ตไซม่อนคาบาเร่ต์ (Similarity: 0.76)
11. Weedland (Similarity: 0.76)
12. Small Place (Similarity: 0.76)
13. La Torcedora (Similarity: 0.72)
14. Z1mplex Mixology Laboratory (Similarity: 0.72)
15. Roots Rock Reggae Bar (Similarity: 0.69)
16. Dibuk House (Similarity: 0.69)
17. Pashtoosh Art Gallery (Similarity: 0.68)
18. Madame Bar (Similarity: 0.66)
19. Coral Premium Departure Lounge (Similarity: 0.65)
20. Patong Inn Art (Similarity: 0.65)


In [11]:
distances

array([[0.        , 0.07130913, 0.11929114, 0.12453446, 0.13514656,
        0.15668114, 0.16946047, 0.1752139 , 0.21428571, 0.23549043,
        0.23856887, 0.24362123, 0.277643  , 0.2816693 , 0.30549601,
        0.31375643, 0.32163491, 0.34452647, 0.34794934, 0.35434416]])

In [12]:
indices

array([[ 94, 245, 336, 214, 237, 194, 232, 328, 219, 131, 115, 168, 207,
        160, 176, 181, 362, 216, 368,  86]])